## 숫자예측 모델(LSTM)

In [14]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [15]:
# 1. 데이터 생성 (랜덤 숫자 시퀀스)
data = np.array([i for i in range(1, 21)]) # 1부터 20까지의 숫자
timesteps = 3 # 타임스텝 길이

In [16]:
x_data = []
y_data = []

In [17]:
# 타임스텝에 따라 입력 데이터와 출력 데이터 생성
for i in range(len(data) - timesteps):
    x_data.append(data[i:i+timesteps]) # 입력(3개의 숫자)
    y_data.append(data[i+timesteps]) # 출력(다음 숫자)
    
x_data = np.array(x_data).reshape(-1, timesteps, 1) # 샘플 수, 타임스텝, 피처 수
y_data = np.array(y_data) # 출력값

In [18]:
# 2. LSTM 모델 정의
model = Sequential([
    LSTM(10, input_shape=(timesteps, 1)), # LSTM 레이어(유닛:10)
    Dense(1) # 출력 레이어
])

In [19]:
model.compile(optimizer='adam', loss='mse')

In [23]:
# 3. 모델 학습
model.fit(x_data, y_data, epochs=1000, verbose=1) # 100번 반복 학습

Epoch 1/1000
1/1 [==============================] - 0s 15ms/step - loss: 136.8218
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 136.4133
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 136.0035
Epoch 4/1000
1/1 [==============================] - 0s 4ms/step - loss: 135.5922
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 135.1790
Epoch 6/1000
1/1 [==============================] - 0s 5ms/step - loss: 134.7637
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 134.3458
Epoch 8/1000
1/1 [==============================] - 0s 10ms/step - loss: 133.9251
Epoch 9/1000
1/1 [==============================] - 0s 4ms/step - loss: 133.5013
Epoch 10/1000
1/1 [==============================] - 0s 3ms/step - loss: 133.0740
Epoch 11/1000
1/1 [==============================] - 0s 19ms/step - loss: 132.6429
Epoch 12/1000
1/1 [==============================] - 0s 13ms/step - loss: 132.2079
Epoch 13/1000
1/1 [==

In [24]:
# 4. 예측 테스트
test_input = np.array([18, 19, 20]).reshape(1, timesteps, 1) # 입력 예시
predicted_value = model.predict(test_input)

In [25]:
# 결과 출력
print("입력 데이터:", test_input.flatten()) # flatten함수: [[1,2],[3,4]] --> [1,2,3,4]
print("예측 값:", predicted_value[0][0])

입력 데이터: [18 19 20]
예측 값: 12.792112
